In [29]:
from collections import defaultdict
import glob
import numpy as np
import os
import re

import pandas as pd
from tqdm import tqdm

In [2]:
# delta y

In [35]:
from tablebench.core.tabular_dataset import CachedDataset
cache_dir = "../tmp"
experiment = "acsfoodstamps"

uids_list = [
    "acsfoodstampsdomain_split_varname_DIVISIONdomain_split_ood_value_06",
]


In [37]:
# list of (uid, split, n_split, ymean) tuples
results = []

for uid in uids_list:
    print(f'processing uid {uid}')
    base_dir = os.path.join(cache_dir, uid)   
    for split in ('id_test', 'ood_test', 'validation'):
        print(f'processing split {split}')
        yvals = list()
        split_files = glob.glob(os.path.join(base_dir, split, '*', '*.csv'))

        for f in tqdm(split_files):
            df = pd.read_csv(f)
            yvals.extend(df[dset.target].values.tolist())
        
        n_split = len(yvals)
        ybar_split = np.array(yvals).mean()
        results.append((uid, split, n_split, ybar_split))
        

processing uid acsfoodstampsdomain_split_varname_DIVISIONdomain_split_ood_value_06
processing split id_test


100%|███████████████████████████████████████████| 23/23 [00:01<00:00, 18.90it/s]


processing split ood_test


100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 15.71it/s]


processing split validation


100%|███████████████████████████████████████████| 24/24 [00:01<00:00, 19.97it/s]


In [38]:
pd.DataFrame(results, columns=['uid', 'split', 'n_split', 'ybar_split'])

,uid,split,n_split,ybar_split
0,acsfoodstampsdomain_split_varname_DIVISIONdoma...,id_test,78628,0.190899
1,acsfoodstampsdomain_split_varname_DIVISIONdoma...,ood_test,48878,0.219874
2,acsfoodstampsdomain_split_varname_DIVISIONdoma...,validation,78627,0.187238
